In [1]:
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

 $r(i,j)$ scalar; = 1;  if user j rated movie i  = 0  otherwise             
$y(i,j)$ scalar; = rating given by user j on movie  i    (if r(i,j) = 1 is defined)   
$\mathbf{w}^{(j)}$vector; parameters for user j  
$b^{(j)}$ scalar; parameter for user j    
$\mathbf{x}^{(i)}$ vector; feature ratings for movie i          
$n_u$ number of users  
$n_m$ number of movies    
$n$ number of features   
$\mathbf{X}$ matrix of vectors $\mathbf{x}^{(i)}$         
$\mathbf{W}$ matrix of vectors $\mathbf{w}^{(j)}$         
$\mathbf{b}$ vector of bias parameters $b^{(j)}$   
$\mathbf{R}$ matrix of elements $r(i,j)$                  

## Recommender Systems
The goal of a collaborative filtering recommender system is to generate two vectors: For each user, a 'parameter vector' that embodies the movie rastes of a user.
For each movie, a feature vector of the same size which embodies some description of the movie. 

The dot product of the two vectors plus the bias term should produce an estimate of the rating the user might give to that movie.

### Importing dataset

In [104]:
# get the total number of unique movies
movie_rows = len(df['movieId'].unique())

In [105]:
# list to add ratings by each user for a particular movie
lst = []

In [106]:
df = pd.read_csv("./dataset/ml-latest-small/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [109]:
# loop to get the rating for a movie from all the user and append it to a list
for each in range(1, movie_rows):
    final_list = np.zeros(shape=(n_u,))
    # get the ratings for a movie by all user
    movie_rating = df[df['movieId'] == each]['rating'].to_numpy()
    # get all the user who has rated the movie 
    userid_in_data = df[df['movieId'] == each]['userId'].to_numpy()

    # if the user is is in the list of user who has rated, get the index
    final_list[np.where(np.isin(userid_generated, userid_in_data ))] = movie_rating

    lst.append(final_list)



In [110]:
# convert the loop to array
Y = np.array(lst)

In [111]:
"""
Here, each row represents a movie and each column represents rating given by a user
"""
Y

array([[4. , 0. , 0. , ..., 2.5, 3. , 5. ],
       [0. , 0. , 0. , ..., 2. , 0. , 0. ],
       [4. , 0. , 0. , ..., 2. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [112]:
R = Y > 1

In [115]:
"""
Converting bool to int. 
Each row represents a movie, and each column represents 1(for if the user has rated) and 0(for if not)

"""
R = R.astype(int)

In [116]:
np.mean(Y, axis=1)

array([1.38196721, 0.61885246, 0.27786885, ..., 0.        , 0.        ,
       0.        ])

In [119]:
Y[0]

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [123]:
f"{np.mean(Y[0, R[0, :].astype(bool)]):0.2f}"

'3.94'

### 4.1 Collaborative filtering cost function

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
The first summation in (1) is "for all $i$, $j$ where $r(i,j)$ equals $1$" and could be written:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularization}
$$

You should now write cofiCostFunc (collaborative filtering cost function) to return this cost.

In [ ]:
## non-vectorised implementation

def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape
    J = 0
    regularised_parameter = 0
    regularised_feature = 0
    ### START CODE HERE ###
    for user_ in range(nu):
        user_cost = 0
        regualrized_parameter_user = 0
        
        for movie_ in range(nm):
            
            user_cost += R[movie_, user_] * np.square(((np.dot(W[user_] , X[movie_]) + b[0][user_]) - (Y[movie_, user_])))
           
            
        for parameter_ in range(len(W[user_])):    
            regualrized_parameter_user += np.square(W[user_][parameter_])
            
        J += user_cost
        
        regularised_parameter += regualrized_parameter_user
    
        
        
        
    for feature_ in range(nm):    
        regularised_feature_movie = np.sum(np.square(X[feature_]))
        regularised_feature += regularised_feature_movie
            
            
            
            
    
    
    ### END CODE HERE ### 
    total_cost = J/2 + ((lambda_/2) * regularised_parameter) + ((lambda_/2) * regularised_feature)

    return total_cost 